In [9]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import datasets, transforms, models

# Setze den Seed für reproduzierbare Ergebnisse
torch.manual_seed(42)

# Definiere die Transformationen für die Bilder
# Wichtig für Transfer Learning: Größe 224x224 und spezielle Normalisierungswerte
image_transforms = transforms.Compose([
    transforms.Resize(size=256),         # Vergrößere die kleinere Seite auf 256px
    transforms.CenterCrop(size=224),     # Schneide einen 224x224 großen Bereich aus der Mitte aus
    transforms.ToTensor(),               # Wandle das Bild in einen PyTorch-Tensor um
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]) # Standard-Normalisierung für ImageNet
])

# Lade den gesamten Datensatz mit den Transformationen
full_dataset = datasets.ImageFolder(root='./data/animals', transform=image_transforms)

# Teile die Daten in einen Trainings- und einen Testsatz auf
# 100 Bilder zum Trainieren, 29 zum Testen
train_set, test_set = torch.utils.data.random_split(full_dataset, [100, 29])

# Erstelle die DataLoader, um die Daten in Batches zu laden
batch_size = 10
train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=len(test_set), shuffle=False) # Lade Testdaten in einem Batch

# Wähle das Gerät (GPU falls verfügbar, sonst CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [11]:
import torch.nn as nn
import torch.nn.functional as F

class CNN(nn.Module):
    # Der Konstruktor, in dem die einzelnen Schichten des Netzwerks definiert werden.
    def __init__(self):
        super(CNN, self).__init__()
        # Erste Faltungsschicht (Convolutional Layer):
        # Nimmt 3-Kanal-Bilder (RGB) auf und erzeugt 10 Feature-Maps mit einem 5x5-Filter.
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=10, kernel_size=5)
        
        # Pooling-Schicht: Verkleinert die Feature-Maps um den Faktor 2 (downsampling).
        # Diese eine Schicht-Definition wird nach beiden Conv-Layern wiederverwendet.
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        
        # Zweite Faltungsschicht:
        # Nimmt die 10 Feature-Maps von conv1 auf und erzeugt 20 neue Feature-Maps.
        self.conv2 = nn.Conv2d(in_channels=10, out_channels=20, kernel_size=5)
        
        # Erste vollvernetzte Schicht (Fully Connected Layer):
        # Nimmt den flachgedrückten Vektor auf (56180 Features) und reduziert ihn auf 50 Features.
        self.fc1 = nn.Linear(in_features=56180, out_features=50)
        
        # Zweite und letzte Schicht (Output Layer):
        # Nimmt die 50 Features auf und gibt die finalen Scores für die 2 Klassen aus.
        self.fc2 = nn.Linear(in_features=50, out_features=2)

    # Die forward-Methode definiert den Datenfluss durch das zuvor definierte Netzwerk.
    # Input x hat die Form [batch_size, 3, 224, 224]
    def forward(self, x):
        # 1. Stufe: Faltung -> Aktivierung -> Pooling
        # Input: [3, 224, 224] -> nach conv1: [10, 220, 220] -> nach pool: [10, 110, 110]
        x = self.pool(F.relu(self.conv1(x)))
        
        # 2. Stufe: Faltung -> Aktivierung -> Pooling
        # Input: [10, 110, 110] -> nach conv2: [20, 106, 106] -> nach pool: [20, 53, 53]
        x = self.pool(F.relu(self.conv2(x)))
        
        # "Flatten": Wandelt die 3D-Feature-Map in einen 1D-Vektor um.
        # Die -1 sorgt dafür, dass die Batch-Größe automatisch beibehalten wird.
        # Input: [batch_size, 20, 53, 53] -> Output: [batch_size, 56180]
        x = x.view(-1, 56180) 
        
        # Datenfluss durch die erste vollvernetzte Schicht mit ReLU-Aktivierung.
        # Input: [batch_size, 56180] -> Output: [batch_size, 50]
        x = F.relu(self.fc1(x))
        
        # Datenfluss durch die finale Ausgabeschicht.
        # Input: [batch_size, 50] -> Output: [batch_size, 2]
        x = self.fc2(x)
        
        # Gibt die finalen Scores (Logits) zurück.
        return x

In [12]:
# Verlustfunktion und eine Beispielfunktion zum Evaluieren
loss_function = nn.CrossEntropyLoss()

def evaluate(model, dataloader):
    model.eval()
    correct = 0
    with torch.no_grad():
        for data, target in dataloader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()
    accuracy = 100. * correct / len(dataloader.dataset)
    print(f'Genauigkeit: {accuracy:.2f}%')
    return accuracy

def train(model, train_loader, epochs=20):
    model.to(device)
    optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
    
    for epoch in range(epochs):
        model.train()
        for batch_idx, (data, target) in enumerate(train_loader):
            data, target = data.to(device), target.to(device)
            optimizer.zero_grad()
            output = model(data)
            loss = loss_function(output, target)
            loss.backward()
            optimizer.step()
        
        print(f'Epoch {epoch+1}/{epochs} - Loss: {loss.item():.4f}')

    print("\nTraining abgeschlossen. Evaluiere auf dem Test-Set:")
    evaluate(model, test_loader)

In [13]:
print("--- Training des CNN von Grund auf ---")
cnn_from_scratch = CNN()
train(cnn_from_scratch, train_loader, epochs=20)

--- Training des CNN von Grund auf ---
Epoch 1/20 - Loss: 0.5391
Epoch 2/20 - Loss: 0.4632
Epoch 3/20 - Loss: 0.0745
Epoch 4/20 - Loss: 0.0954
Epoch 5/20 - Loss: 0.2030
Epoch 6/20 - Loss: 0.0947
Epoch 7/20 - Loss: 0.0375
Epoch 8/20 - Loss: 0.2142
Epoch 9/20 - Loss: 0.4365
Epoch 10/20 - Loss: 0.0303
Epoch 11/20 - Loss: 0.0178
Epoch 12/20 - Loss: 0.0341
Epoch 13/20 - Loss: 0.0132
Epoch 14/20 - Loss: 0.0115
Epoch 15/20 - Loss: 0.0140
Epoch 16/20 - Loss: 0.0001
Epoch 17/20 - Loss: 0.0094
Epoch 18/20 - Loss: 0.0023
Epoch 19/20 - Loss: 0.0036
Epoch 20/20 - Loss: 0.0001

Training abgeschlossen. Evaluiere auf dem Test-Set:
Genauigkeit: 82.76%


In [14]:
# Schritt 1: Ein vor-trainiertes Experten-Modell laden
# Hier wird die ResNet18-Architektur geladen UND mit den Gewichten initialisiert,
# die durch das Training auf dem riesigen ImageNet-Datensatz (Millionen Bilder) gelernt wurden.
# Das Modell ist also bereits ein Experte für die Erkennung allgemeiner Bildmerkmale.
model_resnet = models.resnet18(weights=models.ResNet18_Weights.DEFAULT)

# Schritt 2: Alle gelernten Gewichte "einfrieren"
# Diese Schleife geht durch jeden einzelnen Parameter (Gewichte und Biases) im gesamten Modell...
for param in model_resnet.parameters():
    # ...und teilt PyTorch mit, dass für diesen Parameter keine Gradienten berechnet werden sollen.
    # Das bedeutet, die vor-trainierten Gewichte werden beim neuen Training nicht verändert.
    # Wir nutzen das Modell als festen "Feature Extractor".
    param.requires_grad = False

# Schritt 3: Die letzte Schicht für unsere Aufgabe anpassen
# Die letzte Schicht des originalen ResNet18 war dafür da, 1000 ImageNet-Klassen zu erkennen.
# Wir müssen sie durch eine neue Schicht ersetzen, die nur unsere 2 Klassen kennt.

# Zuerst lesen wir aus, wie viele Eingangs-Merkmale die letzte Schicht (genannt 'fc') erwartet.
# Bei ResNet18 sind das 512.
num_ftrs = model_resnet.fc.in_features

# Jetzt überschreiben wir die alte 'fc'-Schicht mit einer komplett neuen Linearen Schicht.
# Diese neue Schicht nimmt die 512 Features vom Rest des Netzwerks entgegen und hat
# nur noch 2 Ausgänge (einen für jede unserer Klassen, z.B. Delfin und Elefant).
# WICHTIG: Die Parameter dieser neuen Schicht haben standardmäßig `requires_grad=True` und werden somit als einzige trainiert.
model_resnet.fc = nn.Linear(num_ftrs, 2)

1.1%

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /home/datata1/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


100.0%


In [15]:
print("\n--- Training des ResNet18 via Transfer Learning ---")
# Das Training dauert nur einen Moment, da nur eine Schicht trainiert wird.
train(model_resnet, train_loader, epochs=20)


--- Training des ResNet18 via Transfer Learning ---
Epoch 1/20 - Loss: 0.4835
Epoch 2/20 - Loss: 0.1617
Epoch 3/20 - Loss: 0.2018
Epoch 4/20 - Loss: 0.0738
Epoch 5/20 - Loss: 0.0472
Epoch 6/20 - Loss: 0.3996
Epoch 7/20 - Loss: 0.2189
Epoch 8/20 - Loss: 0.0498
Epoch 9/20 - Loss: 0.0146
Epoch 10/20 - Loss: 0.0469
Epoch 11/20 - Loss: 0.1776
Epoch 12/20 - Loss: 0.0304
Epoch 13/20 - Loss: 0.0252
Epoch 14/20 - Loss: 0.0216
Epoch 15/20 - Loss: 0.1558
Epoch 16/20 - Loss: 0.0574
Epoch 17/20 - Loss: 0.1199
Epoch 18/20 - Loss: 0.0384
Epoch 19/20 - Loss: 0.2732
Epoch 20/20 - Loss: 0.0521

Training abgeschlossen. Evaluiere auf dem Test-Set:
Genauigkeit: 96.55%
